# GNSS Trace Comparison Tool

In [1]:
# --- Cell 1: Imports ---
import pandas as pd
from ipyfilechooser import FileChooser
from IPython.display import display
import ipywidgets as widgets

In [ ]:
# --- Cell 2: File Selection UI ---
chooser1 = FileChooser('..', title='Select Trace 1 CSV:')
chooser1.filter_pattern = '*.csv'
chooser1.show_only_dirs = False

chooser2 = FileChooser('..', title='Select Trace 2 CSV:')
chooser2.filter_pattern = '*.csv'
chooser2.show_only_dirs = False

load_button = widgets.Button(description="Load Selected CSV Files")
output = widgets.Output()

def load_files(btn):
    with output:
        output.clear_output()
        try:
            if not chooser1.selected or not chooser2.selected:
                print("❗ Please select BOTH CSV files before loading.")
                return
            df1 = pd.read_csv(chooser1.selected)
            df2 = pd.read_csv(chooser2.selected)
            print("✅ CSV files loaded successfully! Showing first rows below:")
            display(df1.head())
            display(df2.head())
            # Save globally for later cells
            globals()['gnss_trace_1_pd'] = df1
            globals()['gnss_trace_2_pd'] = df2
        except Exception as e:
            print("Error loading files:", e)

load_button.on_click(load_files)

display(widgets.HTML("**Step 1: Use the pickers below. They start in the current server directory.**"))
display(chooser1, chooser2, load_button, output)


HTML(value='**Step 1: Use the pickers below. They start in the current server directory.**')

FileChooser(path='C:\Users\nguye\Documents\GitHub\emtl30kr_gnss_logger', filename='', title='Select Trace 1 CS…

FileChooser(path='C:\Users\nguye\Documents\GitHub\emtl30kr_gnss_logger', filename='', title='Select Trace 2 CS…

Button(description='Load Selected CSV Files', style=ButtonStyle())

Output()

# Data Pre-processing


In [6]:
# --- Cell 3: Data Pre-processing Guarded ---
gnss_trace_1_pd = globals().get('gnss_trace_1_pd', None)
gnss_trace_2_pd = globals().get('gnss_trace_2_pd', None)

if gnss_trace_1_pd is None or gnss_trace_2_pd is None:
    print("❗ Both CSV files must be loaded (Step 1) before preprocessing. Please go back and load the files.")
else:
    # Filter for desired FixTypes
    filtered_fix_types = [
        'fixed-rtk',
        'dead-reckoning',
    ]

    filtered_trace_1_pd = gnss_trace_1_pd[gnss_trace_1_pd['FixType'].isin(filtered_fix_types)]
    filtered_trace_2_pd = gnss_trace_2_pd[gnss_trace_2_pd['FixType'].isin(filtered_fix_types)]

    # Select columns
    filtered_columns = ['GNSSTime', 'Latitude', 'Longitude', 'FixType']
    filtered_trace_1_pd = filtered_trace_1_pd[filtered_columns]
    filtered_trace_2_pd = filtered_trace_2_pd[filtered_columns]
    
    # Store in globals for later cells
    globals()['filtered_trace_1_pd'] = filtered_trace_1_pd
    globals()['filtered_trace_2_pd'] = filtered_trace_2_pd
    print("✅ Pre-processing completed.")
    display(filtered_trace_1_pd.head())
    display(filtered_trace_2_pd.head())


✅ Pre-processing completed.


,GNSSTime,Latitude,Longitude,FixType
0,12:30:13.680000,36.742708,127.117057,fixed-rtk
1,12:30:13.710000,36.742708,127.117057,fixed-rtk
2,12:30:13.750000,36.742708,127.117057,fixed-rtk
3,12:30:13.780000,36.742708,127.117057,fixed-rtk
4,12:30:13.810000,36.742708,127.117057,fixed-rtk


,GNSSTime,Latitude,Longitude,FixType
2,12:30:18.100000,36.742708,127.117057,fixed-rtk
9,12:30:18.130000,36.742708,127.117057,fixed-rtk
13,12:30:18.170000,36.742708,127.117057,fixed-rtk
17,12:30:18.200000,36.742708,127.117057,fixed-rtk
22,12:30:18.230000,36.742708,127.117057,fixed-rtk


In [7]:
# --- Cell 4: Time Overlap Filtering Guarded ---
filtered_trace_1_pd = globals().get('filtered_trace_1_pd', None)
filtered_trace_2_pd = globals().get('filtered_trace_2_pd', None)

if filtered_trace_1_pd is None or filtered_trace_2_pd is None:
    print("❗ Pre-processing must be completed before time range adjustment. Please run the previous cell.")
elif filtered_trace_1_pd.empty or filtered_trace_2_pd.empty:
    print("❗ One or both filtered trace DataFrames are empty. Check your fix type filter or data.")
else:
    start_time = max(filtered_trace_1_pd['GNSSTime'].min(), filtered_trace_2_pd['GNSSTime'].min())
    end_time = min(filtered_trace_1_pd['GNSSTime'].max(), filtered_trace_2_pd['GNSSTime'].max())
    filtered_trace_1_pd = filtered_trace_1_pd[
        (filtered_trace_1_pd['GNSSTime'] >= start_time) & (filtered_trace_1_pd['GNSSTime'] <= end_time)
    ]
    filtered_trace_2_pd = filtered_trace_2_pd[
        (filtered_trace_2_pd['GNSSTime'] >= start_time) & (filtered_trace_2_pd['GNSSTime'] <= end_time)
    ]
    globals()['filtered_trace_1_pd'] = filtered_trace_1_pd
    globals()['filtered_trace_2_pd'] = filtered_trace_2_pd
    print(f"✅ Time filtering complete. Start: {start_time}, End: {end_time}")
    print(f"Trace 1 rows: {len(filtered_trace_1_pd)}, Trace 2 rows: {len(filtered_trace_2_pd)}")


✅ Time filtering complete. Start: 12:30:18.100000, End: 12:31:15.390000
Trace 1 rows: 1142, Trace 2 rows: 1737


# Visualize GNSS Traces on a Map

In [ ]:
# (Cell 5)
import folium
import numpy as np

filtered_trace_1_pd = globals().get('filtered_trace_1_pd', None)
filtered_trace_2_pd = globals().get('filtered_trace_2_pd', None)

if filtered_trace_1_pd is None or filtered_trace_2_pd is None or filtered_trace_1_pd.empty or filtered_trace_2_pd.empty:
    print("❗ Cannot plot. Please complete prior steps.")
else:
    # Center map at average location
    center_lat = np.mean(list(filtered_trace_1_pd['Latitude']) + list(filtered_trace_2_pd['Latitude']))
    center_lon = np.mean(list(filtered_trace_1_pd['Longitude']) + list(filtered_trace_2_pd['Longitude']))

    m = folium.Map(location=[center_lat, center_lon], zoom_start=13)

    # Trace 1
    folium.PolyLine(
        list(zip(filtered_trace_1_pd['Latitude'], filtered_trace_1_pd['Longitude'])),
        color="blue", weight=3, opacity=0.7, tooltip="Trace 1"
    ).add_to(m)

    # Trace 2
    folium.PolyLine(
        list(zip(filtered_trace_2_pd['Latitude'], filtered_trace_2_pd['Longitude'])),
        color="red", weight=3, opacity=0.7, tooltip="Trace 2"
    ).add_to(m)

    # Start and end markers
    folium.Marker(
        [filtered_trace_1_pd.iloc[0]['Latitude'], filtered_trace_1_pd.iloc[0]['Longitude']],
        popup="Trace 1 Start", icon=folium.Icon(color='blue')
    ).add_to(m)
    folium.Marker(
        [filtered_trace_1_pd.iloc[-1]['Latitude'], filtered_trace_1_pd.iloc[-1]['Longitude']],
        popup="Trace 1 End", icon=folium.Icon(color='blue', icon='flag')
    ).add_to(m)
    folium.Marker(
        [filtered_trace_2_pd.iloc[0]['Latitude'], filtered_trace_2_pd.iloc[0]['Longitude']],
        popup="Trace 2 Start", icon=folium.Icon(color='red')
    ).add_to(m)
    folium.Marker(
        [filtered_trace_2_pd.iloc[-1]['Latitude'], filtered_trace_2_pd.iloc[-1]['Longitude']],
        popup="Trace 2 End", icon=folium.Icon(color='red', icon='flag')
    ).add_to(m)

    m
